# Multi-Layer Perceptron

Bu notebook'te MNIST verisetini kullanarak basit bir çok katmanlı perceptron yapay sinir ağı eğiteceğiz. 

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.layers import fully_connected
from datetime import datetime

/Users/elvan/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Değişkenleri ve placeholderları oluşturalım:

In [2]:
n_inputs = 28*28 # MNIST 28x28 görsellerden oluşuyor. Bunları 1D Tensöre çeviriyoruz.
n_hid1 = 250
n_hid2 = 150
n_outputs = 10
learning_rate = 0.0001
n_epochs = 100
batch_size = 50
now = datetime.utcnow().strftime("%Y%m%d&H%M%S")
root_logdir = "tf_logs_mlp"
logdir = "{}/run-{}".format(root_logdir, now)

X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X")
y = tf.placeholder(tf.int64, shape = (None), name = "y")

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data')

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


Değişkenleri tanımladıktan sonra Graph'ımızı oluşturabiliriz. Bu noktada "name scope" konseptinden bahsetmek gerekiyor. "Name scope"ları kullanarak aynı yapıya sahip işlemleri bir araya gruplayabiliriz. Bu işlem her ne kadar opsiyonel olsa da, hem graph'ınızın daha temiz olmasını sağlayacak hem de kodu kendi içerisinde bölümlere ayırarak okunabilirliğini arttıracaktır.

Network scope'unda 250X150X10 yapısına sahip bir Multilayer Perceptron modeli oluşturduk. Gizli katmanlarımızın aktivasyon fonksiyonları, fully_connected fonksiyonunun default aktivasyon fonksiyonu olan RELU. RELU fonksiyonu isim olarak çok büyük olsa da aslında çok basit bir fonksiyon. 

$$RELU(x) = max(0, x)$$

Kısacası RELU fonksiyonu girdisi 0'dan büyükse girdisini, değilse 0 döndürüyor. Eğer aktivasyon fonksiyonunu değiştirmek istiyorsanız 'activation_fn' parametresiyle bunu belirleyebilirsiniz.

In [3]:
# Network Scope'u
with tf.name_scope("dnn"):
    hid1 = fully_connected(X, n_hid1, scope = "hid1")
    hid2 = fully_connected(hid1, n_hid2, scope = "hid2") # Hidden 1'in output'u, Hidden 2'nin input'u oluyor
    logits = fully_connected(hid2, n_outputs, scope = "outputs", activation_fn = None)

Loss scope'unda, modelin nasıl öğrenileceğini belirttik. Modeli "Cross Entropy" fonksiyonunu optimize ederek öğreneceğimizi belirttik.

Multi-class sınıflandırmayı normalde burada olduğu gibi 'Soft-max' fonksiyonuyla yapıyoruz. Ancak dikkat ettiyseniz bu fonksiyonu network scope'unda aktivasyon fonksiyonu olarak tanımlamadık. Bunun sebebi kullandığımız "sparse_softmax_cross_entropy_with_logits" kayıp fonksiyonunun içinde zaten softmax işlemi bulunması. Bu fonksiyonun güzel tarafı, logitlerin 0 olduğu durumda bile çalışacak şekilde sparse olarak yazılmış olması. Bu sayede potansiyel hataların önüne geçmiş oluyoruz.

In [4]:
# Loss Scope'u
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
    loss = tf.reduce_mean(xentropy, name = "loss")

Öğrenme scope'unda TF'a kullanacağımız optimizasyon yöntemini ve minimize edeceğimiz değeri veriyoruz.

In [5]:
# Train Scope'u
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

Değerlendirme scope'unda ise modelin başarısını nasıl değerlendireceğimizi TF'a bildiriyoruz. Bu kısımda doğru tahmin yüzdesi olan 'accuracy' metriğini kullanmayı seçtim. Bunu yapabilmek için 'in_top_k' fonksiyonunu kullanabiliriz. Bu fonksiyon size, doğru sınıfın yaptığınız sınıf tahminin ilk 'k' değer içinde olup olmadığını verir. k'yı 1 seçtiğimizde ise ilk tahminimizin doğru tahmin olup olmadığını öğreniriz.

In [6]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

Son olarak değişkenlerimizi ve işlemlerimizi başlatarak, loglarımızı oluşturuyoruz. Bu noktada, önceki notebook'tan farklı olarak, modelimizi daha sonra kullanmak üzere kaydedeceğiz.

In [7]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

acc_summary = tf.summary.scalar('Train_Accuracy', accuracy)
acc_summary_test = tf.summary.scalar('Test_Accuracy', accuracy)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

Kodu artık koşturabiliriz.

In [8]:
with tf.Session() as sess:
    init.run() # Değişkenleri başlat
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size): # batch sayısını modulo aritmetiği ile hesapla
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict = {X: X_batch, y:y_batch})
            if(iteration % 10 == 0):
                summary_str = acc_summary.eval(feed_dict = {X: X_batch, y:y_batch})
                summary_str_test = acc_summary_test.eval(feed_dict = {X: mnist.test.images, y:mnist.test.labels})
                step = epoch*(mnist.train.num_examples // batch_size) + iteration
                file_writer.add_summary(summary_str, step)
                file_writer.add_summary(summary_str_test, step)
            
        acc_train = accuracy.eval(feed_dict = {X: X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict = {X: mnist.test.images, y:mnist.test.labels})
        if(epoch % 10 ==0):
            print("Epoch:", epoch, "Train Accuracy:", acc_train, "Test Accuracy:", acc_test)
    save_path = saver.save(sess, "./final_mnist.ckpt")

Epoch: 0 Train Accuracy: 0.08 Test Accuracy: 0.1065
Epoch: 10 Train Accuracy: 0.8 Test Accuracy: 0.7154
Epoch: 20 Train Accuracy: 0.76 Test Accuracy: 0.8086
Epoch: 30 Train Accuracy: 0.82 Test Accuracy: 0.8455
Epoch: 40 Train Accuracy: 0.88 Test Accuracy: 0.8642
Epoch: 50 Train Accuracy: 0.82 Test Accuracy: 0.8768
Epoch: 60 Train Accuracy: 0.84 Test Accuracy: 0.8853
Epoch: 70 Train Accuracy: 0.88 Test Accuracy: 0.8898
Epoch: 80 Train Accuracy: 0.82 Test Accuracy: 0.8947
Epoch: 90 Train Accuracy: 0.94 Test Accuracy: 0.8985


Graph ve performans grafikleri aşağıdadır:

Graph: ![alt text](images/graph_mlp.png "Graph")
Performans: ![alt text](images/tr_test_acc_mlp.png "Performans")